Cell 1: Install and import

In [ ]:
!pip install numpy pandas matplotlib yfinance scipy

import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from scipy.optimize import minimize


Cell 2: Download historical prices

In [ ]:
tickers = ['AAPL', 'MSFT', 'GOOG', 'TSLA']
data = yf.download(tickers, start="2022-01-01", end="2023-12-31")['Adj Close']
data = data.dropna()

returns = data.pct_change().dropna()
mean_returns = returns.mean()
cov_matrix = returns.cov()


Cell 3: Define portfolio stats

In [ ]:
def portfolio_performance(weights, mean_returns, cov_matrix):
    returns = np.dot(weights, mean_returns)
    std = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    return returns, std


Cell 4: Optimize for Max Sharpe

In [ ]:
def negative_sharpe(weights, mean_returns, cov_matrix, risk_free_rate=0.01):
    ret, std = portfolio_performance(weights, mean_returns, cov_matrix)
    return -(ret - risk_free_rate) / std

num_assets = len(tickers)
bounds = tuple((0, 1) for asset in range(num_assets))
constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
init_guess = num_assets * [1. / num_assets,]

opt_result = minimize(negative_sharpe, init_guess,
                      args=(mean_returns, cov_matrix),
                      method='SLSQP',
                      bounds=bounds,
                      constraints=constraints)


Cell 5: Show Optimal Portfolio

In [ ]:
optimal_weights = opt_result.x
for ticker, weight in zip(tickers, optimal_weights):
    print(f"{ticker}: {weight:.2%}")

ret, std = portfolio_performance(optimal_weights, mean_returns, cov_matrix)
sharpe_ratio = (ret - 0.01) / std

print(f"\nExpected Return: {ret:.2%}")
print(f"Volatility: {std:.2%}")
print(f"Sharpe Ratio: {sharpe_ratio:.2f}")


Cell 6: Plot Efficient Frontier

In [ ]:
results = {'Returns': [], 'Volatility': [], 'Sharpe': []}
weights_list = []

for _ in range(5000):
    w = np.random.dirichlet(np.ones(num_assets), size=1).flatten()
    r, s = portfolio_performance(w, mean_returns, cov_matrix)
    results['Returns'].append(r)
    results['Volatility'].append(s)
    results['Sharpe'].append((r - 0.01) / s)
    weights_list.append(w)

df = pd.DataFrame(results)
plt.scatter(df['Volatility'], df['Returns'], c=df['Sharpe'], cmap='viridis', alpha=0.5)
plt.scatter(std, ret, c='red', s=50, label='Optimal Portfolio')
plt.colorbar(label='Sharpe Ratio')
plt.xlabel("Volatility")
plt.ylabel("Expected Return")
plt.title("Efficient Frontier")
plt.legend()
plt.show()


Cell 7: Save Optimal Weights

In [ ]:
opt_df = pd.DataFrame({
    'Ticker': tickers,
    'Weight': optimal_weights
})
opt_df.to_csv("optimal_portfolio_weights.csv", index=False)
print("✅ Saved as optimal_portfolio_weights.csv")
